# Tutorial: solving electronic structure problem

### Overview

This is overview architecture of this demo

![electronic_structure_problem](images/electronic_structure_problem.png)

### Implementation

Let's see how we can implement this.
For full implementation [see here](./source_files/electronic_structure_problem.py).

```python
from quantum_serverless import run_qiskit_remote, get


@run_qiskit_remote()
def ground_state_solve(
    molecule: Molecule,
    initial_point: Union[List[float], np.ndarray],
    options: Optional[Options] = None,
    service: Optional[Union[QiskitRuntimeService, Dict[str, Any]]] = None,
):
    driver = PySCFDriver.from_molecule(molecule=molecule, ...)
    es_problem = ElectronicStructureProblem(driver, ...)
    ansatz = ...
    operator = qubit_converter.convert(es_problem.second_q_ops(), ...)
    optimizer = COBYLA(maxiter=500)
    ...
    
    with Session(service=service) as session:
        estimator = Estimator(session=session, options=options)

        vqe = EstimatorVQE(
            estimator=estimator,
            circuit=ansatz,
            optimizer=optimizer,
            init_point=initial_point,
        )

        vqe_result = vqe.compute_minimum_eigenvalue(operator)
       
    return vqe_result.optimal_value


def electronic_structure_problem(
    molecules: List[Molecule],
    initial_points: Optional[List[List[float]]] = None,
    service: Optional[QiskitRuntimeService] = None,
    backends: Optional[List[IBMQBackend]] = None,
):
    ...
    function_references = [
        hardware_efficient_ansatz(
            molecule=molecule,
            initial_point=initial_point,
            service=service,
        )
        for molecule, initial_point in zip(molecules, initial_points)
    ]

    return get(function_references)
```

### Execution

In [9]:
from quantum_serverless import QuantumServerless, Program

In [36]:
serverless = QuantumServerless.load_configuration("./serverless_config.json") # config with docker setup

In [41]:
program = Program(
    name="electronic_structure",
    entrypoint="electronic_structure_problem.py",
    working_dir="./source_files/",
    description="Electronic structure program",
    dependencies=["pyscf==1.6.3"]
)

job = serverless.run_program(program)
job

<Job | fqs_e1fa4e3c-6bf4-4567-9d36-93488ad87c34>

In [ ]:
print("\n".join(job.logs().split("\n")[-4:-1]))

Energies:  [-0.9848168378887934, -0.9094747516508735, -0.9811709328709944]
Shifts:  [-6.609784771130337, -6.781951626949085, -6.870414678425508]
Energy + shift:  [-7.59460160901913, -7.691426378599958, -7.851585611296502]
